In [ ]:
from vqa_logger import logger 
import IPython


In [ ]:
from classes.vqa_model_predictor import VqaModelPredictor
from common.DAL import get_models_data_frame, get_model
from evaluate.VqaMedEvaluatorBase import VqaMedEvaluatorBase
df_models = get_models_data_frame()
df_models

In [ ]:
model_id = max(df_models.id)
mp = VqaModelPredictor(model_id)
mp

In [ ]:
mp.df_validation.head(2)

In [ ]:
code = get_highlited_function_code(mp.predict,remove_comments=False)
IPython.display.display(code)

In [ ]:
df_data = mp.df_validation
df_predictions = mp.predict(mp.df_validation)
df_predictions.head()

In [ ]:
df_predictions.describe()

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
idx = 42
image_names = m.image_name.values
image_name = image_names[idx]

df_image = df_predictions[df_predictions.image_name == image_name]
print(f'Result: {set(df_image.prediction)}')

image_path = df_image.path.values[0]
df_image

In [ ]:
from IPython.display import Image
Image(filename = image_path, width=400, height=400)

In [ ]:
df_image = df_data[df_data.image_name == image_name]
image_prediction = mp.predict(df_image)

In [ ]:
validation_prediction.head(2)

## Evaluating the Model

In [ ]:
validation_prediction = df_predictions
predictions = validation_prediction.prediction.values
ground_truth = validation_prediction.answer.values
results = VqaMedEvaluatorBase.get_all_evaluation(predictions=predictions, ground_truth=ground_truth)
print(f'Got results of\n{results}')